# 作业３：设计并训练KNN算法对图片进行分类。

example1:

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

k=7
test_num=int(input('请输入需要测试的数据数量：'))

#加载TFRecord训练集的数据
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["/home/srhyme/ML project/DS/train.tfrecords"])    
_, example = reader.read(filename_queue)  
features = tf.parse_single_example(
    example,features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'pixels': tf.FixedLenFeature([], tf.int64),  
        'label': tf.FixedLenFeature([], tf.int64),
    })
train_images = tf.decode_raw(features['image_raw'], tf.uint8)
train_labels = tf.cast(features['label'], tf.int32)
train_pixels = tf.cast(features['pixels'], tf.int32)

#加载TFRecord测试集的数据
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["/home/srhyme/ML project/DS/test.tfrecords"])    
_, example = reader.read(filename_queue)  
features = tf.parse_single_example(
    example,features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'pixels': tf.FixedLenFeature([], tf.int64),  
        'label': tf.FixedLenFeature([], tf.int64),
    })
test_images = tf.decode_raw(features['image_raw'], tf.uint8)
test_labels = tf.cast(features['label'], tf.int32)
test_pixels = tf.cast(features['pixels'], tf.int32)

tri_list=[]
tei_list=[]
trl_list=[]
tel_list=[]

#转换TFRecord里面的类型格式
with tf.Session() as sess:  
    coord = tf.train.Coordinator()  
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    for i in range(sess.run(train_pixels)):
        image,label=sess.run([train_images,train_labels])
        tri_list.append(image)
        trl=np.zeros((1,10))
        trl[0][label]=1
        trl_list.append(trl[0])
    train_labels=np.array(trl_list)
    train_images=np.array(tri＿list)
    print('训练集已加载完毕')

    for i in range(test_num):
        image,label=sess.run([test_images,test_labels])
        tei_list.append(image)
        tel=np.zeros((1,10))
        tel[0][label]=1
        tel_list.append(tel[0])
    test_labels=np.array(tel_list)
    test_images=np.array(tei＿list)
    print('测试集已加载完毕')
    
sess.close()

x_train = tf.placeholder(tf.float32)
x_test = tf.placeholder(tf.float32)
y_train = tf.placeholder(tf.float32)

# 欧式距离
euclidean_distance = tf.sqrt(tf.reduce_sum(tf.square(x_train - x_test), 1))
# 计算最相近的k个样本的索引
_, nearest_index = tf.nn.top_k(-euclidean_distance, k)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    predicted_num = 0
    # 对每个图片进行预测
    for i in range(test_images.shape[0]):
        # 最近k个样本的标记索引
        nearest_index_res = sess.run(
            nearest_index, 
            feed_dict={
                x_train: train_images,
                y_train: train_labels,
                x_test: test_images[i]})
        # 最近k个样本的标记
        nearest_label = []
        for j in range(k):
            nearest_label.append(list(train_labels[nearest_index_res[j]]))

        predicted_class = sess.run(tf.argmax(tf.reduce_sum(nearest_label, 0), 0))
        true_class = sess.run(tf.argmax(test_labels[i]))
        if predicted_class == true_class:
            predicted_num += 1
        
        if i % 100 == 0:
            print('step is %d accuracy is %.4f' % (i, predicted_num / (i+1)))
    print('accuracy is %.4f' % (predicted_num / test_num))

请输入需要测试的数据数量：300
训练集已加载完毕
测试集已加载完毕
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer_1/input_producer_1_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer_1, input_producer_1/RandomShuffle)]]
step is 0 accuracy is 1.0000
step is 100 accuracy is 0.8515
step is 200 accuracy is 0.8756
accuracy is 0.8867


example2:

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from collections import Counter


#训练集数据的引入
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["/home/srhyme/ML project/DS/train.tfrecords"])    
_, example = reader.read(filename_queue)  
features = tf.parse_single_example(
    example,features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'pixels': tf.FixedLenFeature([], tf.int64),  
        'label': tf.FixedLenFeature([], tf.int64),
    })
train_images = tf.decode_raw(features['image_raw'], tf.uint8)
train_labels = tf.cast(features['label'], tf.int32)
train_pixels = tf.cast(features['pixels'], tf.int32) 


#测试集数据的引入
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["/home/srhyme/ML project/DS/test.tfrecords"])    
_, example = reader.read(filename_queue)  
features = tf.parse_single_example(
    example,features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'pixels': tf.FixedLenFeature([], tf.int64),  
        'label': tf.FixedLenFeature([], tf.int64),
    })
test_images = tf.decode_raw(features['image_raw'], tf.uint8)
test_labels = tf.cast(features['label'], tf.int32)
test_pixels = tf.cast(features['pixels'], tf.int32)


#设置变量
testnum=int(input('请输入需要测试的数据集数量:'))
k=5
correct_probability=testnum

with tf.Session() as sess:  
    coord = tf.train.Coordinator()  
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    test_num=sess.run(test_pixels)
    train_num=sess.run(train_pixels)


    #生成一个训练标签的列表,方便索引
    c_labels=[]
    for n in range(train_num):
        train_label=sess.run(train_labels)
        c_labels.append(train_label)
    #生成一个测试标签的列表,方便索引
    g_labels=[]
    for n in range(test_num):
        test_label=sess.run(test_labels)
        g_labels.append(test_label)


    for i in range(testnum):#测试集数量
        test_image=sess.run(test_images)
        #生成每个测试集的距离列表
        min_label=[]
        for j in range(train_num):#训练集数量
            train_image=sess.run(train_images)
            euclidean_distance =np.sqrt(np.sum(np.square(train_image - test_image)))
            min_label.append(-euclidean_distance)
            

        #生成最近k个点的位置
        min_labels=tf.constant([min_label])
        _, nearest_index = tf.nn.top_k(min_labels, k)
        #生成一个最近k点标签列表
        nearest_label=[]
        near=nearest_index
        for m in range(k):
            nearest_label.append(c_labels[sess.run(near[0,m])])#在训练标签中找到该位置的标签
        

        #生成该测试集经过knn训练后拟合的标签
        nearset_dict=Counter(nearest_label)
        
        key_list=[]
        value_list=[]
        for key,value in nearset_dict.items():
            key_list.append(key)
            value_list.append(value)
        max_value=max(value_list)
        get_value_index = value_list.index(max_value)
        guess = key_list[get_value_index]
        #判断正确率
        correct=g_labels[i]
        if correct != guess:
            correct_probability=correct_probability - 1

print('正确率为',(correct_probability/testnum))

请输入需要测试的数据集数量:10
正确率为 INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer_2/input_producer_2_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer_2, input_producer_2/RandomShuffle)]]0.7

